In [14]:
import pandas as pd
from thunderpack import ThunderReader
from tqdm import tqdm
import tarfile
import zipfile
import random
import os
import shutil
import re
tqdm.pandas()

## Goal
Filter notes to only include those with Brain MRI or CT Head
- Search through notes with regex for keywords 
    - If any are there, keep it, else, drop it

In [15]:
admission_notes_df = pd.read_csv('2_admission_notes_df.csv')
print(len(admission_notes_df))
admission_notes_df.head()

19168


,BDSPPatientID,AdmissionDate,ICD,NoteDate,NoteTitle
0,150005970,2020-12-09,I25.2,2020-12-22,Notes_1129864534_17441726072_20201222.txt
1,150787076,2018-02-04,R33.9,2018-01-27,Notes_1130645711_1863357139_20180127.txt
2,151344919,2022-05-12,I48.91,2022-04-30,Notes_1131203861_3479700929_20220430.txt
3,151076568,2012-06-26,621.0,2012-06-10,Notes_1130935590_2521399596_20120610.txt
4,150994453,2018-04-11,C61,2018-04-12,Notes_1130852937_2253966504_20180412.txt


In [16]:
def convert_tar_to_zip(tar_path, zip_path):
    with tarfile.open(tar_path, 'r') as tar:
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for member in tqdm(tar.getmembers()):
                file_obj = tar.extractfile(member)
                if file_obj is not None:
                    zipf.writestr(member.name, file_obj.read())

# This will extract the year from the file name
def extract_year(file_name):
    match = re.search(r'_(\d{8})\.txt$', file_name)
    if match:
        return match.group(1)[:4]
    return None

def read_file_from_zip(base_path, file_name):
    year = extract_year(file_name)
    if year:
        zip_path = f'{base_path}/bidmc_notes_{year}.zip'
        try:
            with zipfile.ZipFile(zip_path, 'r') as zipf:
                with zipf.open(f'{year}/{file_name}') as file:
                    content = file.read()
                    return content.decode('utf-8', errors = 'ignore')
        except (KeyError, FileNotFoundError):
            return None
    return None



In [6]:
year = 2010
for i in tqdm(range(0, 15)):
    tar_file_path = f'/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_{year + i}.tar'
    zip_file_path = f'/home/jsearle/bigDrive/NAX/zipNotes/BIDMC/bidmc_notes_{year + i}.zip'
    convert_tar_to_zip(tar_file_path, zip_file_path)

100%|██████████| 15/15 [57:48<00:00, 231.21s/it]


In [17]:
base_path = '/home/jsearle/bigDrive/NAX/zipNotes/BIDMC'

pattern1 = re.compile(r'(MRI|MRA|MR ANGIO|CT|CTA|CT ANGIO)\s*(of)*\s*(the)*\s*(brw|brain|brn|neck|head|hed)', re.IGNORECASE)
pattern2 = re.compile(r'(brw|brain|brn|neck|head|hed)\s*(of)*\s*(the)*\s*(MRI|MRA|MR ANGIO|CT|CTA|CT ANGIO)', re.IGNORECASE)

def read_and_filter(note_title, base_path):
    note_text = read_file_from_zip(base_path, note_title)
    if note_text and (pattern1.search(note_text) or pattern2.search(note_text)):
        return note_text
    else:
        return None


chunk_size = 4000
start = 0
end = min(start + chunk_size, len(admission_notes_df))
chunk = admission_notes_df.iloc[start:end].copy()


# Apply the filtering function
chunk['text'] = chunk['NoteTitle'].progress_apply(lambda x: read_and_filter(x, base_path))

# Drop rows where 'text' is None
filtered_chunk = chunk.dropna(subset=['text'])

filtered_chunk.head()
print(len(filtered_chunk))

100%|██████████| 4000/4000 [12:37:09<00:00, 11.36s/it]  

484


In [18]:
# Not enough in first chunk, so doing another 500 (need at least 500 matches at the end)

chunk_size = 500
start = 3501
end = min(start + chunk_size, len(admission_notes_df))
chunk = admission_notes_df.iloc[start:end].copy()

# Apply the filtering function
chunk['text'] = chunk['NoteTitle'].progress_apply(lambda x: read_and_filter(x, base_path))

# Drop rows where 'text' is None
filtered_chunk2 = chunk.dropna(subset=['text'])

filtered_chunk2.head()
print(len(filtered_chunk2))

100%|██████████| 500/500 [1:40:57<00:00, 12.11s/it]

64


In [21]:
# Not enough in first 2 chunks, so doing another 500 (need at least 500 matches at the end)

chunk_size = 500
start = 4001
end = min(start + chunk_size, len(admission_notes_df))
chunk = admission_notes_df.iloc[start:end].copy()

# Apply the filtering function
chunk['text'] = chunk['NoteTitle'].progress_apply(lambda x: read_and_filter(x, base_path))

# Drop rows where 'text' is None
filtered_chunk2 = chunk.dropna(subset=['text'])

filtered_chunk2.head()
print(len(filtered_chunk2))

100%|██████████| 500/500 [1:35:18<00:00, 11.44s/it]

53


In [19]:
new_filtered_chunk = pd.concat([filtered_chunk, filtered_chunk2], ignore_index=True)
print(len(new_filtered_chunk))

548


In [22]:
new_filtered_chunk2 = pd.concat([new_filtered_chunk, filtered_chunk2], ignore_index=True)
print(len(new_filtered_chunk2))

601


In [23]:
new_filtered_chunk2.to_csv('3_cpt_filtered_chunk2.csv', index=False, header=True, sep=',', na_rep='NA')